In [20]:
from urllib.parse import urlencode
import requests
base_url = "https://m.weibo.cn/api/container/getIndex?"

headers = {
  'Host': 'm.weibo.cn',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0',
  'X-Requested-With': 'XMLHttpRequest',
  'Referer': 'https://m.weibo.cn/u/2830678475?luicode=10000011&lfid=1005052830678475',
}

def get_page(since_id):
  params = {
    'luicode': '10000011',
    'lfid': '1005052830678475',
    'type': 'uid',
    'value': '2830678475',
    'containerid': '1076032830678475', 
  }
  if since_id:
    params['since_id'] = since_id
  url = base_url + urlencode(params)
  try:
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
      return response.json()
  except requests.ConnectionError as e:
    print('Error', e.args)

In [21]:
from pyquery import PyQuery as pq

def parse_page(json):
  if json:
    items = json.get('data').get('cards')
    for item in items:
      item = item.get('mblog')
      weibo = {}
      weibo['id'] = item.get('id')
      weibo['text'] = pq(item.get('text')).text()
      weibo['attitudes'] = item.get('attitudes_count')
      weibo['comments'] = item.get('comments_count')
      weibo['reposts'] = item.get('reposts_count')
      yield weibo

# TODO: try/except
def parse_since_id(json):
  if json:
    return json.get('data').get('cardlistInfo').get('since_id')

In [23]:
import csv
if __name__ == '__main__':
  json = get_page(0)
  # print(json)
  since_id = parse_since_id(json)
  print(since_id)
  results = parse_page(json)
  with open('weibo.csv', 'w', encoding='utf-8') as csvfile:
    fieldnames = ['id', 'text', 'attitudes', 'comments', 'reposts']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
      writer.writerow(result)
  while since_id:
    json = get_page(since_id)
    since_id = parse_since_id(json)
    results = parse_page(json)
    print("since_id:", since_id)
    with open('weibo.csv', 'a', encoding='utf-8') as csvfile:
      fieldnames = ['id', 'text', 'attitudes', 'comments', 'reposts']
      writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
      # writer.writeheader()
      for result in results:
        writer.writerow(result)

3968761201544157
since_id: 3963629110332444
since_id: 3963451573351990
since_id: 3961759339772693
since_id: 3961759247773225
since_id: 3923451872320948
since_id: 3922021186601482
since_id: 3920240553518416
since_id: 3919684296954068
since_id: 3919366179433890
since_id: 3917938946242737
since_id: 3917358768995870
since_id: 3913359315132600
since_id: 3911936665828432
since_id: 3911076251394753
since_id: 3910182134313729
since_id: 3907914693463523
since_id: 3907203809304192
since_id: 3904736761354733
since_id: 3900392678904634
since_id: 3899670067850170
since_id: 3896788429395769


AttributeError: 'NoneType' object has no attribute 'get'